# Homework on word embeddings

## Task 1

1) Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно, и с помощью модели, взятой с rusvectores (например вот этой - http://vectors.nlpl.eu/repository/20/180.zip). Обучите 2 модели по определению перефразирования на получившихся векторах и проверьте, что работает лучше. 

Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие. 
!!!! ВАЖНО: Оценивать модели нужно с помощью кросс-валидации (в семинаре не кросс-валидация)! Метрика - f1.

In [1]:
import pandas as pd
from lxml import html
import numpy as np
from pymystem3 import Mystem
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
from razdel import tokenize as razdel_tokenize
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

### Normalization

In [2]:
class Normilizer():

    def __init__(self, morph_type):
        
        self.morpho = MorphAnalyzer() 
        self.cashe = {}
        self.stops = set(stopwords.words('russian'))
        
    
    def normalize(self, text) -> list:
        """
            returns a normalized text with POS tags mapped to udpipe
        """
        
        words = self.tokenize(text)
        
        res=[]
        
        mapping = self.generate_mapping('data/ru-rnc.map.txt')

        for word in words:
            if not word or word in self.stops:# skip stop words
                continue 
            elif word in self.cashe: # check cashed first
                res.append(self.cashe[word])
            else:
                r=self.morpho.parse(word)[0]
                lemma = r.normal_form
                pos = r.tag.POS
                try:
                    pos = mapping[pos]
                    res.append(lemma+'_'+pos)
                    self.cashe[word]=lemma+'_'+pos
                except:
                    res.append('Error')
                
        return res

    
    def tokenize(self, text) -> str:
        """
            tokenizes a text and keeps only alphanumeric tokens
        """
        punct = punctuation+'«»—…“”*№–'
        
        tokens = [token.text.strip(punct).lower() for token in list(razdel_tokenize(text))]
        tokens = [token for token in tokens if token.isalnum()]

        return tokens
    
    
    def generate_mapping(self, path) -> dict:
        """ 
            generates mapping of PoS tags to map mystem and udpipe tags:
            Mapping was update for pymorphy2
        """
        mapping = {}

        for line in open(path):
            ms, ud = line.strip('\n').split()
            mapping[ms] = ud
            
        return mapping

In [3]:
data = open('data/wiki_data.txt', encoding='utf8').read().splitlines()
norm = Normilizer('pymorphy2')
norm.normalize('Обучить классификатор парафразов на предобученной модели вам нужно будет дома')

['обучить_VERB',
 'классификатор_NOUN',
 'парафраз_NOUN',
 'предобученный_ADJ',
 'модель_NOUN',
 'нужно_X',
 'дом_NOUN']

In [4]:
%%time
data_norm = [norm.normalize(text) for text in data]

Wall time: 1min 28s


In [5]:
len(data_norm)

20002

### Modelling

my embedding

In [6]:
w2v = gensim.models.Word2Vec([text for text in data_norm], size=50, sg=1)

In [7]:
w2v.most_similar('полиция_NOUN')

<ipython-input-7-05aad1014560>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v.most_similar('полиция_NOUN')


[('полицейский_ADJ', 0.8340840935707092),
 ('полицейский_NOUN', 0.8244892358779907),
 ('преступник_NOUN', 0.7986570596694946),
 ('милиция_NOUN', 0.7955402135848999),
 ('охранник_NOUN', 0.789502739906311),
 ('жандарм_NOUN', 0.786804735660553),
 ('подозревать_ADJ', 0.786204993724823),
 ('гестапо_NOUN', 0.7759683728218079),
 ('бандит_NOUN', 0.7743021845817566),
 ('спецназ_NOUN', 0.7564249634742737)]

In [8]:
w2v

rusvectores http://vectors.nlpl.eu/repository/20/180.zip

In [9]:
rusvec = gensim.models.KeyedVectors.load_word2vec_format('data/models/180/model.bin', binary=True)

In [10]:
rusvec.most_similar('полиция_NOUN')

[('полиция_PROPN', 0.78785240650177),
 ('полицейский_ADJ', 0.7626974582672119),
 ('полицейский_NOUN', 0.6821430921554565),
 ('жандармерия_NOUN', 0.6472468376159668),
 ('жандарм_NOUN', 0.6468209624290466),
 ('городовый_ADJ', 0.6110862493515015),
 ('сыскный_ADJ', 0.6072753667831421),
 ('жандармский_ADJ', 0.6051692962646484),
 ('агент_NOUN', 0.5971013307571411),
 ('градоначальник_NOUN', 0.5876316428184509)]

Vectorizing

In [11]:
class Vectorizer():
    """ word2vec vectorizer """
    
    def __init__(self, model, dim):
        self.model = model # Gensim w2v model
        self.dim = dim
        

    def get_embedding(self, text):
        """ transforms a text into a vector using w2v model """
        
        text = text.split()
        words = Counter(text) # cashe words
        total = len(text)
        vectors = np.zeros((len(words), self.dim))

        for i,word in enumerate(words):
            try:
                v = self.model[word]
                vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
            except (KeyError, ValueError):
                continue
        
        
        if vectors.any():
            vector = np.average(vectors, axis=0)
        else:
            vector = np.zeros((self.dim))
        
        return vector

In [12]:
corpus_xml = html.fromstring(open('data/paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])

data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [13]:
data['text_1_norm'] = data['text_1'].apply(norm.normalize)
data['text_2_norm'] = data['text_2'].apply(norm.normalize)
data['text_1_norm'] = data['text_1_norm'].apply(" ".join)
data['text_2_norm'] = data['text_2_norm'].apply(" ".join)

In [14]:
data.head(2)

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский_NOUN разрешить_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешить_VERB стрелять...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право_NOUN полицейский_ADJ проникновение_NOUN ...,правило_NOUN внесудебный_ADJ проникновение_NOU...


In [15]:
data.label.value_counts()

0     2957
-1    2582
1     1688
Name: label, dtype: int64

In [16]:
y = data['label']
y.shape

(7227,)

In [17]:
dim=50

W2V = Vectorizer(w2v, dim)

X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = W2V.get_embedding(text)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = W2V.get_embedding(text)
    
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

<ipython-input-11-75724c73e890>:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = self.model[word]


In [18]:
X_text_w2v.shape

(7227, 100)

In [19]:
X_text_w2v[0]

array([-0.02467497, -0.0424685 ,  0.00706543,  0.07053129,  0.02111456,
        0.00026692,  0.02947436, -0.00262682, -0.02419937, -0.03342579,
        0.06407263,  0.03926686,  0.00204358, -0.03341437, -0.01235045,
       -0.02330916,  0.04435799, -0.00258285,  0.06149818,  0.02719655,
       -0.01207639, -0.00268261, -0.05692433, -0.02179623,  0.01853627,
       -0.09280529,  0.06385348, -0.03415607, -0.03120847, -0.04610864,
        0.02290655,  0.06502387,  0.00248277,  0.01359456,  0.00874802,
       -0.04827983, -0.06036325,  0.02155151,  0.07595971, -0.01363526,
        0.04501745, -0.00981721,  0.0408394 , -0.00950802, -0.08377997,
       -0.06442852, -0.02202226,  0.00171277,  0.0041399 , -0.01984598,
       -0.01384089, -0.07207043, -0.01958374,  0.06150077,  0.02150663,
        0.01519106,  0.02775176, -0.00300476, -0.0246584 , -0.05242948,
        0.05648939,  0.03133879,  0.02379792, -0.01279469, -0.00908745,
       -0.02772011,  0.04417225,  0.00640621,  0.04720146,  0.04

In [20]:
dim=50

RV = Vectorizer(rusvec, dim)

X_text_1_rv = np.zeros((len(data['text_1_norm']), dim))
X_text_2_rv = np.zeros((len(data['text_2_norm']), dim))
for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_rv[i] = RV.get_embedding(text)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_rv[i] = RV.get_embedding(text)
    
X_text_rv = np.concatenate([X_text_1_rv, X_text_2_rv], axis=1)

In [21]:
X_text_rv.shape

(7227, 100)

In [22]:
X_text_rv[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

comparing models

In [23]:
from sklearn.model_selection import cross_val_score

In [24]:
clf1 = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf2 = LogisticRegression(C=10000, class_weight='balanced')

In [25]:
%%capture --no-stdout _
print(cross_val_score(clf1, X_text_w2v, y, scoring="f1_micro"))
print(cross_val_score(clf2, X_text_w2v, y, scoring="f1_micro"))

[0.4571231  0.48547718 0.49065744 0.35709343 0.38961938]
[0.38589212 0.41355463 0.42698962 0.35363322 0.3349481 ]


In [26]:
%%capture --no-stdout _
print(cross_val_score(clf1, X_text_rv, y, scoring="f1_micro"))
print(cross_val_score(clf2, X_text_rv, y, scoring="f1_micro"))

[0.35684647 0.35684647 0.23321799 0.40899654 0.23391003]
[0.35684647 0.35684647 0.35778547 0.35778547 0.35709343]


## Task 2

Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары).

Постройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).  Попробуйте улучить метрику, изменив параметры в методах векторизации.
!!УТОЧНЕНИЕ: модель нужно обучить сразу на всех 5 близостях, а не по 1 модели на каждой близости!


In [39]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

5 vector types

In [26]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

TfidfVectorizer(max_df=0.4, max_features=1000, min_df=3)

In [59]:
svd = TruncatedSVD(200)

X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_norm']))

In [60]:
nmf = NMF(100)

X_text_1_nmf = nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.fit_transform(tfidf.transform(data['text_2_norm']))

X_text_nmf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

In [61]:
w2v = gensim.models.Word2Vec([text for text in data_norm], size=50, sg=1)

dim=50

W2V = Vectorizer(w2v, dim)

X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = W2V.get_embedding(text)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = W2V.get_embedding(text)

<ipython-input-13-d1a9c2abcf3c>:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = self.model[word]


In [62]:
fast_text = gensim.models.FastText([text for text in data_norm], size=50, min_n=4, max_n=8) 

dim=50

FT = Vectorizer(fast_text, dim)

X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = FT.get_embedding(text)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = FT.get_embedding(text)

<ipython-input-13-d1a9c2abcf3c>:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = self.model[word]


In [ ]:
RV = Vectorizer(rusvec, dim)

X_text_1_rv = np.zeros((len(data['text_1_norm']), dim))
X_text_2_rv = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_rv[i] = RV.get_embedding(text)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_rv[i] = RV.get_embedding(text)

In [93]:
cosine_similarity(X_text_1_svd[0:1], X_text_2_svd[0:1])

array([[0.07642255]])

In [90]:
def calculate_row_similarity(A, B):
    """ calculates similarity between each row of matrix A and matrix B """
    output = []
    for row in zip(A, B):
        # add a value only
        output.extend(cosine_similarity([row[0]], [row[1]])[0])
        
    return output

In [99]:
svd_sim = calculate_row_similarity(X_text_1_svd, X_text_2_svd)
nmf_sim = calculate_row_similarity(X_text_1_nmf, X_text_2_nmf)
w2v_sim = calculate_row_similarity(X_text_1_w2v, X_text_2_w2v)
ft_sim = calculate_row_similarity(X_text_1_ft, X_text_2_ft)
rv_sim = calculate_row_similarity(X_text_1_rv, X_text_2_rv)

similarities = pd.DataFrame({
    'svd': svd_sim,
    "nmf": nmf_sim,
    'w2v': w2v_sim,
    'ft' : ft_sim,
    'rv' : rv_sim,
})

similarities.head()

,svd,nmf,w2v,ft,rv
0,0.076423,0.000123,0.914622,0.897895,0.0
1,0.231177,0.597571,0.930156,0.865366,0.0
2,-0.098049,0.044629,0.950981,0.792569,0.0
3,-0.206381,0.000000,0.792817,0.575355,0.0
4,-0.211944,0.000741,0.924721,0.684357,0.0


Классификаторы 1

In [100]:
clf1 = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf2 = LogisticRegression(C=10000, class_weight='balanced')

In [103]:
%%capture --no-stdout _
print(cross_val_score(clf1, similarities, y, scoring="f1_micro"))
print(cross_val_score(clf2, similarities, y, scoring="f1_micro"))

[0.5373444  0.56431535 0.61453287 0.46228374 0.44844291]
[0.51452282 0.53526971 0.58062284 0.42698962 0.43598616]


Классификаторы 2: Изменение гиперпараметров (увеличение или уменьшение) не дает ощутимой разницы в результатах

In [112]:
clf1 = RandomForestClassifier(n_estimators=50, max_depth=5, min_samples_leaf=10,
                             class_weight='balanced')
clf2 = LogisticRegression(C=1000, class_weight='balanced')

In [113]:
%%capture --no-stdout _
print(cross_val_score(clf1, similarities, y, scoring="f1_micro"))
print(cross_val_score(clf2, similarities, y, scoring="f1_micro"))

[0.54149378 0.56639004 0.61591696 0.45743945 0.44636678]
[0.51452282 0.53526971 0.58062284 0.42629758 0.43598616]


Оба классификатора, обученные на косинусной близости, дают более высокий результат в сравнении с классификаторами обученными только на эмбеддингах ~ +0.1


**Изменение методов векторизации**

In [126]:
tfidf = TfidfVectorizer(min_df=5, max_df=0.7, max_features=2000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

TfidfVectorizer(max_df=0.7, max_features=2000, min_df=5)

In [127]:
svd = TruncatedSVD(300)

X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_norm']))

In [128]:
nmf = NMF(150)

X_text_1_nmf = nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.fit_transform(tfidf.transform(data['text_2_norm']))

X_text_nmf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

In [117]:
w2v = gensim.models.Word2Vec([text for text in data_norm], size=100, sg=1)

dim=100

W2V = Vectorizer(w2v, dim)

X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = W2V.get_embedding(text)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = W2V.get_embedding(text)

<ipython-input-13-d1a9c2abcf3c>:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = self.model[word]


In [118]:
fast_text = gensim.models.FastText([text for text in data_norm], size=100, min_n=4, max_n=8) 

dim=100

FT = Vectorizer(fast_text, dim)

X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = FT.get_embedding(text)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = FT.get_embedding(text)

<ipython-input-13-d1a9c2abcf3c>:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = self.model[word]


In [119]:
RV = Vectorizer(rusvec, 100)

X_text_1_rv = np.zeros((len(data['text_1_norm']), dim))
X_text_2_rv = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_rv[i] = RV.get_embedding(text)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_rv[i] = RV.get_embedding(text)

In [129]:
svd_sim = calculate_row_similarity(X_text_1_svd, X_text_2_svd)
nmf_sim = calculate_row_similarity(X_text_1_nmf, X_text_2_nmf)
w2v_sim = calculate_row_similarity(X_text_1_w2v, X_text_2_w2v)
ft_sim = calculate_row_similarity(X_text_1_ft, X_text_2_ft)
rv_sim = calculate_row_similarity(X_text_1_rv, X_text_2_rv)

similarities = pd.DataFrame({
    'svd': svd_sim,
    "nmf": nmf_sim,
    'w2v': w2v_sim,
    'ft' : ft_sim,
    'rv' : rv_sim,
})

similarities.head()

,svd,nmf,w2v,ft,rv
0,-0.067069,0.000165,0.874152,0.890109,0.0
1,0.009539,0.030172,0.898566,0.869398,0.0
2,-0.012295,0.002107,0.919779,0.773450,0.0
3,-0.106305,0.000304,0.783218,0.574517,0.0
4,-0.073217,0.080916,0.904459,0.658687,0.0


сравннение

In [130]:
clf1 = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf2 = LogisticRegression(C=10000, class_weight='balanced')

In [131]:
%%capture --no-stdout _
print(cross_val_score(clf1, similarities, y, scoring="f1_micro"))
print(cross_val_score(clf2, similarities, y, scoring="f1_micro"))

[0.52697095 0.56984786 0.60415225 0.47543253 0.4615917 ]
[0.5055325  0.54771784 0.5799308  0.43183391 0.43252595]


In [132]:
clf1 = RandomForestClassifier(n_estimators=50, max_depth=5, min_samples_leaf=10,
                             class_weight='balanced')
clf2 = LogisticRegression(C=1000, class_weight='balanced')

In [133]:
%%capture --no-stdout _
print(cross_val_score(clf1, similarities, y, scoring="f1_micro"))
print(cross_val_score(clf2, similarities, y, scoring="f1_micro"))

[0.52766252 0.56777317 0.60276817 0.46989619 0.45051903]
[0.5055325  0.54771784 0.5799308  0.43183391 0.43252595]


Изменение параметров не дает заметной разницы (+-0.03)